In [1]:
import torch
import torch.nn as nn

In [2]:
# determenistic mode
torch.manual_seed(0)

# 1ый эксперемент

In [ ]:
# просто посмотреть на то, расчитываются ли градиенты для слоёв

In [3]:
# data
random_input = torch.randn(3,3)
random_output = torch.randn(3,3)

# loss_fn 
loss_fn = nn.MSELoss()

In [4]:
# model
l1 = nn.Linear(3, 10, bias=False)
l2 = nn.Linear(10, 3, bias=False)

In [7]:
params_before = {"l1":list(l1.parameters()), "l2": list(l2.parameters())}
params_before

{'l1': [Parameter containing:
  tensor([[-0.3370,  0.4963,  0.2576],
          [ 0.2798,  0.0304, -0.2960],
          [ 0.0977, -0.5391, -0.4172],
          [-0.2976,  0.3643,  0.3385],
          [-0.2561, -0.0208,  0.3693],
          [ 0.5740,  0.2291,  0.0780],
          [ 0.3871, -0.3399,  0.1076],
          [-0.4476, -0.4002, -0.2982],
          [ 0.2612,  0.2322, -0.3420],
          [ 0.1744,  0.3169, -0.0729]], requires_grad=True)],
 'l2': [Parameter containing:
  tensor([[ 0.0121,  0.0733,  0.1962,  0.3036, -0.2437, -0.1159,  0.1243,  0.2620,
            0.2752,  0.2790],
          [ 0.0629, -0.2750,  0.0291, -0.1978, -0.2947,  0.2810,  0.2404, -0.3154,
            0.0592, -0.0533],
          [-0.0520, -0.1448,  0.1216, -0.1873,  0.1159,  0.1599,  0.2264,  0.1182,
           -0.3130, -0.2051]], requires_grad=True)]}

In [8]:
# перед backward pass попробуем проверить, есть ли какие-то градиенты 

In [9]:
l1.weight.grad

In [10]:
l2.weight.grad

In [ ]:
# как можно заметить, слои не имеют расчитанных градиентов - это ожидаемо и логично т.к. мы ещё не делали backward pass

In [ ]:
random_input -> l1 -> y -> l2 -> z

In [11]:
y = l1(random_input)
z = l2(y)

In [12]:
y

tensor([[-1.2263,  1.0672,  1.2176, -1.3031, -1.1930,  0.6473,  0.4619,  0.0775,
          1.0796,  0.3346],
        [-1.0901,  0.5401,  1.2236, -1.0377, -0.6394, -0.0313,  0.4383,  0.5967,
          0.3750, -0.1427],
        [ 0.0947,  0.3512, -0.1123, -0.0583, -0.3863,  0.3674, -0.2061, -0.3014,
          0.5459,  0.3884]], grad_fn=<MmBackward0>)

In [13]:
z

tensor([[ 0.5904,  0.5887, -0.0261],
        [ 0.3850,  0.1503,  0.3242],
        [ 0.1927,  0.1919, -0.3774]], grad_fn=<MmBackward0>)

In [14]:
loss = loss_fn(z, random_output)

In [15]:
loss

tensor(0.8857, grad_fn=<MseLossBackward0>)

In [16]:
loss.backward()

In [17]:
l1.weight.grad

tensor([[ 0.0264,  0.0383, -0.0247],
        [-0.0659, -0.0393,  0.0764],
        [ 0.1136, -0.1166, -0.2098],
        [ 0.0758, -0.0779, -0.1477],
        [-0.2226, -0.0544,  0.3072],
        [ 0.0471,  0.0490, -0.0446],
        [ 0.1562, -0.0779, -0.2535],
        [ 0.0247, -0.2391, -0.1292],
        [ 0.1477,  0.0655, -0.1987],
        [ 0.1139, -0.0187, -0.1801]])

In [18]:
l2.weight.grad

tensor([[-0.5701,  0.3902,  0.6048, -0.5750, -0.4459,  0.1400,  0.2197,  0.1771,
          0.3504,  0.0406],
        [-0.0769,  0.2262,  0.0434, -0.1339, -0.2443,  0.2390, -0.0056, -0.1564,
          0.2972,  0.1837],
        [-0.2971,  0.0596,  0.3418, -0.2518, -0.0787, -0.1009,  0.1489,  0.2317,
         -0.0247, -0.1245]])

In [ ]:
# после запуска backward pass - оба слоя имеют расчитанные градиенты!

# 2ой эксперемент

In [38]:
# model
l1 = nn.Linear(3, 10, bias=False)
l2 = nn.Linear(10, 3, bias=False)

In [40]:
# заморозим веса 2ого слоя!
for param in l2.parameters():
    param.requires_grad = False

In [41]:
params_before = {"l1":list(l1.parameters()), "l2": list(l2.parameters())}
params_before

{'l1': [Parameter containing:
  tensor([[-0.2297, -0.5397, -0.4349],
          [ 0.2502, -0.3419,  0.0828],
          [ 0.1843,  0.0409, -0.3743],
          [ 0.5521, -0.3357,  0.4749],
          [-0.4593, -0.1389,  0.3141],
          [-0.2359,  0.4850, -0.3973],
          [-0.4848, -0.2603,  0.0933],
          [ 0.5316, -0.2756,  0.2065],
          [-0.1447, -0.1252,  0.4245],
          [-0.4474,  0.0613,  0.5429]], requires_grad=True)],
 'l2': [Parameter containing:
  tensor([[-0.0435,  0.2455, -0.0974,  0.2546, -0.3059, -0.0456, -0.0555,  0.1025,
            0.1241,  0.2428],
          [-0.0471, -0.0125,  0.2166, -0.0856,  0.2772, -0.2105, -0.0342, -0.0170,
            0.1411,  0.2162],
          [-0.0501, -0.2620,  0.1567,  0.0946,  0.1270, -0.1951,  0.2035,  0.2995,
            0.0274, -0.2954]])]}

In [42]:
# перед backward pass попробуем проверить, есть ли какие-то градиенты 

In [43]:
l1.weight.grad

In [44]:
l2.weight.grad

In [ ]:
# как можно заметить, слои не имеют расчитанных градиентов - это ожидаемо и логично т.к. мы ещё не делали backward pass

In [ ]:
random_input -> l1 -> y -> l2 -> z

In [45]:
y = l1(random_input)
z = l2(y)

In [46]:
y

tensor([[ 0.7521,  0.3054,  1.0876, -0.0853, -1.3513,  0.3598, -0.8741,  0.4502,
         -1.1113, -1.8905],
        [ 1.0630,  0.3971,  0.5839,  0.0138, -0.5497, -0.1044, -0.1238,  0.3123,
         -0.5402, -1.0802],
        [-0.2320, -0.2451,  0.3778, -0.4002, -0.5275,  0.5971, -0.4809, -0.1651,
         -0.4687, -0.5196]], grad_fn=<MmBackward0>)

In [47]:
z

tensor([[-0.1905, -0.7901,  0.2876],
        [-0.1195, -0.3710,  0.2586],
        [-0.2291, -0.3011, -0.0930]], grad_fn=<MmBackward0>)

In [48]:
loss = loss_fn(z, random_output)

In [49]:
loss

tensor(0.5536, grad_fn=<MseLossBackward0>)

In [50]:
loss.backward()

In [51]:
l1.weight.grad

tensor([[-0.0032,  0.0123,  0.0076],
        [-0.0067,  0.0402,  0.0280],
        [-0.0371,  0.0504,  0.0759],
        [ 0.0691, -0.1403, -0.1506],
        [-0.0838,  0.1415,  0.1759],
        [ 0.0103,  0.0048, -0.0185],
        [ 0.0334, -0.0843, -0.0814],
        [ 0.0663, -0.1575, -0.1537],
        [-0.0111,  0.0155,  0.0268],
        [-0.0647,  0.1500,  0.1581]])

In [52]:
l2.weight.grad

In [ ]:
# как можно заметить, при заморозке 2ого слоя, градиенты ко 2ому слою не поступают, НО для 1ого они также поступают
# -> заморозка весов действительно помогает, если нужно заморозить какие-то слои, чтобы они не обновлялись без помехи
# для обновления других весов

# Посмотрим на то, как веса обновляются + добавим optimizer

In [ ]:
# Список экспериментов:
# 1) 2 слоя, никто не заморожён, оба подаём оптимизатору 

# 2) 2 слоя, никто не заморожен, но подаём оптимизатору только 2ой слой l2
# 3) 2 слоя, никто не заморожен, но подаём оптимизатору только 1ый слой l1

# 4) 2 слоя, заморожён 2ой слой l2, подаём оба слоя оптимизатору 
# 5) 2 слоя, заморожён 1ый слой l1, подаём оба слоя оптимизаотру
# 6) 2 слоя, оба заморожёны - l1 и l2, подаём оба слоя оптимизатору

# 7) 2 слоя, заморожён 2ой слой l2, подаём только слой l2 оптимизатору


### 1) 2 слоя, никто не заморожён, оба подаём оптимизатору 


In [213]:
# model
l1 = nn.Linear(3, 5, bias=False)
l2 = nn.Linear(5, 3, bias=False)

In [214]:
params_before = {"l1": list(l1.parameters())[0].detach().clone(), "l2": list(l2.parameters())[0].detach().clone()}
params_before

{'l1': tensor([[ 0.4741,  0.2236,  0.4864],
         [-0.1979, -0.3185,  0.4965],
         [ 0.2407,  0.5542, -0.2412],
         [-0.3707, -0.0677, -0.5436],
         [ 0.2263,  0.4258,  0.1386]]),
 'l2': tensor([[-0.0442,  0.2217, -0.2839,  0.4375, -0.4447],
         [-0.4284, -0.1057,  0.3653,  0.0448,  0.1718],
         [-0.3278,  0.1631, -0.0500,  0.1793,  0.3158]])}

In [215]:
optimizer = torch.optim.SGD([{"params": l1.parameters(), "lr": 10}, {"params": l2.parameters(), "lr": 10}])
optimizer

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 10
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0

Parameter Group 1
    dampening: 0
    differentiable: False
    foreach: None
    lr: 10
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [190]:
# перед backward pass попробуем проверить, есть ли какие-то градиенты 

In [216]:
l1.weight.grad

In [217]:
l2.weight.grad

In [ ]:
random_input -> l1 -> y -> l2 -> z

In [220]:
y = l1(random_input)
z = l2(y)

In [221]:
y

tensor([[-0.3948, -1.2931,  0.7336,  0.6331, -0.0782],
        [-0.6533, -0.4614, -0.1270,  0.6230, -0.5270],
        [ 0.0288, -0.7038,  0.7350,  0.1848,  0.3485]], grad_fn=<MmBackward0>)

In [222]:
z

tensor([[-0.1658,  0.5887, -0.0294],
        [ 0.4695,  0.2196,  0.0905],
        [-0.4401,  0.3987, -0.0178]], grad_fn=<MmBackward0>)

In [223]:
loss = loss_fn(z, random_output)

In [224]:
loss

tensor(0.7546, grad_fn=<MseLossBackward0>)

In [225]:
loss.backward()

In [226]:
l1.weight.grad

tensor([[-0.1849, -0.0043,  0.2783],
        [ 0.0101, -0.1861, -0.0809],
        [ 0.0809,  0.2568, -0.0320],
        [ 0.1109, -0.2391, -0.2468],
        [-0.0104,  0.1426,  0.0544]])

In [227]:
l2.weight.grad

tensor([[-0.2170, -0.1162, -0.0906,  0.1939, -0.2030],
        [ 0.0301, -0.4012,  0.3757,  0.0844,  0.1573],
        [-0.1526,  0.0028, -0.1311,  0.1150, -0.1626]])

In [228]:
optimizer.step()
optimizer.zero_grad()

In [136]:
# посмотрим на параметры слоёв, после backward step

In [229]:
params_after = {"l1": list(l1.parameters())[0].detach().clone(), "l2": list(l2.parameters())[0].detach().clone()}
params_after

{'l1': tensor([[ 2.3234,  0.2664, -2.2963],
         [-0.2992,  1.5424,  1.3058],
         [-0.5687, -2.0137,  0.0793],
         [-1.4798,  2.3236,  1.9243],
         [ 0.3299, -1.0004, -0.4057]]),
 'l2': tensor([[ 2.1254,  1.3839,  0.6225, -1.5016,  1.5856],
         [-0.7293,  3.9064, -3.3920, -0.7993, -1.4014],
         [ 1.1981,  0.1353,  1.2610, -0.9710,  1.9423]])}

In [230]:
params_before

{'l1': tensor([[ 0.4741,  0.2236,  0.4864],
         [-0.1979, -0.3185,  0.4965],
         [ 0.2407,  0.5542, -0.2412],
         [-0.3707, -0.0677, -0.5436],
         [ 0.2263,  0.4258,  0.1386]]),
 'l2': tensor([[-0.0442,  0.2217, -0.2839,  0.4375, -0.4447],
         [-0.4284, -0.1057,  0.3653,  0.0448,  0.1718],
         [-0.3278,  0.1631, -0.0500,  0.1793,  0.3158]])}

In [231]:
params_after["l1"] == params_before["l1"], params_after["l2"] == params_before["l2"]

(tensor([[False, False, False],
         [False, False, False],
         [False, False, False],
         [False, False, False],
         [False, False, False]]),
 tensor([[False, False, False, False, False],
         [False, False, False, False, False],
         [False, False, False, False, False]]))

In [ ]:
# как можно заметить, веса каждого из слоёв действительно изменились

### 2) 2 слоя, никто не заморожен, но подаём оптимизатору только 2ой слой l2

In [232]:
# model
l1 = nn.Linear(3, 5, bias=False)
l2 = nn.Linear(5, 3, bias=False)

In [233]:
params_before = {"l1": list(l1.parameters())[0].detach().clone(), "l2": list(l2.parameters())[0].detach().clone()}
params_before

{'l1': tensor([[ 0.2510, -0.0491, -0.0356],
         [-0.3621, -0.2088,  0.3752],
         [-0.2315,  0.3586, -0.2289],
         [-0.1345,  0.0123, -0.5298],
         [-0.0058, -0.0582, -0.0519]]),
 'l2': tensor([[-0.0894,  0.3526,  0.3300, -0.3031,  0.2077],
         [-0.3508, -0.3807,  0.1367,  0.0066, -0.2086],
         [-0.4313, -0.1731,  0.1494, -0.4139, -0.4345]])}

In [234]:
optimizer = torch.optim.SGD([{"params": l2.parameters(), "lr": 10}])
optimizer

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 10
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [190]:
# перед backward pass попробуем проверить, есть ли какие-то градиенты 

In [235]:
l1.weight.grad

In [236]:
l2.weight.grad

In [ ]:
random_input -> l1 -> y -> l2 -> z

In [237]:
y = l1(random_input)
z = l2(y)

In [238]:
y

tensor([[ 0.4786, -1.3142,  0.0368,  0.9435,  0.1212],
        [ 0.2457, -0.5041, -0.2003,  0.6512,  0.1324],
        [ 0.0856, -0.5909,  0.3718,  0.3371, -0.0138]], grad_fn=<MmBackward0>)

In [239]:
z

tensor([[-0.7548,  0.3184, -0.4167],
        [-0.4357,  0.0550, -0.3757],
        [-0.1984,  0.2508, -0.0127]], grad_fn=<MmBackward0>)

In [240]:
loss = loss_fn(z, random_output)

In [241]:
loss

tensor(0.4437, grad_fn=<MseLossBackward0>)

In [242]:
loss.backward()

In [245]:
l1.weight.grad

# как можно заметить, grad для слоя l1 расчитались т.к. requires_grad=True!

tensor([[-0.0152, -0.0220,  0.0207],
        [-0.1045, -0.1452,  0.1032],
        [-0.0195, -0.0285,  0.0161],
        [ 0.0936,  0.1272, -0.0851],
        [-0.0071, -0.0135,  0.0119]])

In [246]:
l2.weight.grad

tensor([[-0.0205,  0.0976, -0.0481, -0.0368,  0.0039],
        [ 0.0558, -0.2574,  0.1215,  0.1018, -0.0086],
        [-0.0331,  0.1476, -0.0677, -0.0538,  0.0055]])

In [247]:
optimizer.step()
optimizer.zero_grad()

In [136]:
# посмотрим на параметры слоёв, после backward step

In [248]:
params_after = {"l1": list(l1.parameters())[0].detach().clone(), "l2": list(l2.parameters())[0].detach().clone()}
params_after

{'l1': tensor([[ 0.2510, -0.0491, -0.0356],
         [-0.3621, -0.2088,  0.3752],
         [-0.2315,  0.3586, -0.2289],
         [-0.1345,  0.0123, -0.5298],
         [-0.0058, -0.0582, -0.0519]]),
 'l2': tensor([[ 0.1157, -0.6238,  0.8114,  0.0650,  0.1687],
         [-0.9088,  2.1933, -1.0786, -1.0111, -0.1225],
         [-0.1008, -1.6490,  0.8266,  0.1239, -0.4896]])}

In [249]:
params_before

{'l1': tensor([[ 0.2510, -0.0491, -0.0356],
         [-0.3621, -0.2088,  0.3752],
         [-0.2315,  0.3586, -0.2289],
         [-0.1345,  0.0123, -0.5298],
         [-0.0058, -0.0582, -0.0519]]),
 'l2': tensor([[-0.0894,  0.3526,  0.3300, -0.3031,  0.2077],
         [-0.3508, -0.3807,  0.1367,  0.0066, -0.2086],
         [-0.4313, -0.1731,  0.1494, -0.4139, -0.4345]])}

In [250]:
params_after["l1"] == params_before["l1"], params_after["l2"] == params_before["l2"]

(tensor([[True, True, True],
         [True, True, True],
         [True, True, True],
         [True, True, True],
         [True, True, True]]),
 tensor([[False, False, False, False, False],
         [False, False, False, False, False],
         [False, False, False, False, False]]))

In [ ]:
# 1) Градиенты для l1 всё равно расчитались, что ожидаемо т.к. requires_grad = True
# 2) Обновление весов произошло только для 2ого слоя l2 т.к. именно он и был передан в оптимизатор
# так что веса 1ого слоя l1 не будут изменяться, хоть и будут расчитываться градиенты

### 3) 2 слоя, никто не заморожен, но подаём оптимизатору только 1ый слой l1

In [251]:
# model
l1 = nn.Linear(3, 5, bias=False)
l2 = nn.Linear(5, 3, bias=False)

In [252]:
params_before = {"l1": list(l1.parameters())[0].detach().clone(), "l2": list(l2.parameters())[0].detach().clone()}
params_before

{'l1': tensor([[ 0.0732, -0.4159, -0.5058],
         [-0.2224, -0.0930, -0.5246],
         [-0.3855,  0.0379, -0.5444],
         [-0.0811, -0.3728, -0.4002],
         [ 0.2269, -0.4444, -0.4375]]),
 'l2': tensor([[ 0.2550,  0.0836,  0.4073, -0.1195, -0.2160],
         [-0.0414,  0.0023,  0.3411, -0.4455, -0.0318],
         [ 0.1978, -0.2463, -0.2272,  0.2161, -0.4285]])}

In [253]:
optimizer = torch.optim.SGD([{"params": l1.parameters(), "lr": 10}])
optimizer

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 10
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [254]:
# перед backward pass попробуем проверить, есть ли какие-то градиенты 

In [255]:
l1.weight.grad

In [256]:
l2.weight.grad

In [ ]:
random_input -> l1 -> y -> l2 -> z

In [257]:
y = l1(random_input)
z = l2(y)

In [258]:
y

tensor([[ 1.3370,  0.8276,  0.5810,  0.8564,  1.4334],
        [ 1.2001,  0.7081,  0.5012,  0.9180,  1.2229],
        [ 0.0448,  0.2097,  0.2678, -0.0573,  0.0338]], grad_fn=<MmBackward0>)

In [259]:
z

tensor([[ 0.2348, -0.2824, -0.5005],
        [ 0.1956, -0.3250, -0.3765],
        [ 0.1376,  0.1144, -0.1305]], grad_fn=<MmBackward0>)

In [260]:
loss = loss_fn(z, random_output)

In [261]:
loss

tensor(0.5901, grad_fn=<MseLossBackward0>)

In [262]:
loss.backward()

In [263]:
l1.weight.grad

# как можно заметить, grad для слоя l1 расчитались т.к. requires_grad=True!

tensor([[ 0.0526, -0.1361, -0.1318],
        [ 0.0765,  0.0299, -0.1010],
        [ 0.1832,  0.0836, -0.2422],
        [-0.0784, -0.2123,  0.0373],
        [ 0.0061,  0.1417,  0.0485]])

In [264]:
l2.weight.grad

tensor([[ 0.4704,  0.2836,  0.2005,  0.3359,  0.4893],
        [-0.2801, -0.1348, -0.0777, -0.2397, -0.2822],
        [-0.0225, -0.0397, -0.0407,  0.0182, -0.0324]])

In [265]:
optimizer.step()
optimizer.zero_grad()

In [266]:
# посмотрим на параметры слоёв, после backward step

In [267]:
params_after = {"l1": list(l1.parameters())[0].detach().clone(), "l2": list(l2.parameters())[0].detach().clone()}
params_after

{'l1': tensor([[-0.4531,  0.9448,  0.8118],
         [-0.9878, -0.3923,  0.4854],
         [-2.2175, -0.7979,  1.8777],
         [ 0.7026,  1.7499, -0.7736],
         [ 0.1664, -1.8615, -0.9227]]),
 'l2': tensor([[ 0.2550,  0.0836,  0.4073, -0.1195, -0.2160],
         [-0.0414,  0.0023,  0.3411, -0.4455, -0.0318],
         [ 0.1978, -0.2463, -0.2272,  0.2161, -0.4285]])}

In [268]:
params_before

{'l1': tensor([[ 0.0732, -0.4159, -0.5058],
         [-0.2224, -0.0930, -0.5246],
         [-0.3855,  0.0379, -0.5444],
         [-0.0811, -0.3728, -0.4002],
         [ 0.2269, -0.4444, -0.4375]]),
 'l2': tensor([[ 0.2550,  0.0836,  0.4073, -0.1195, -0.2160],
         [-0.0414,  0.0023,  0.3411, -0.4455, -0.0318],
         [ 0.1978, -0.2463, -0.2272,  0.2161, -0.4285]])}

In [269]:
params_after["l1"] == params_before["l1"], params_after["l2"] == params_before["l2"]

(tensor([[False, False, False],
         [False, False, False],
         [False, False, False],
         [False, False, False],
         [False, False, False]]),
 tensor([[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]]))

In [ ]:
# 1) Градиенты для l2 всё равно расчитались, что ожидаемо т.к. requires_grad = True
# 2) Обновление весов произошло только для 1ого слоя l1 т.к. именно он и был передан в оптимизатор
# так что веса 2ого слоя l2 не будут изменяться, хоть и будут расчитываться градиенты для него

### 4) 2 слоя, заморожён 2ой слой l2, подаём оба слоя оптимизатору 

In [274]:
# model
l1 = nn.Linear(3, 5, bias=False)
l2 = nn.Linear(5, 3, bias=False)

In [275]:
# замораживаем параметры 2ого слоя l2

for param in l2.parameters():
    param.requires_grad = False

In [276]:
params_before = {"l1": list(l1.parameters())[0].detach().clone(), "l2": list(l2.parameters())[0].detach().clone()}
params_before

{'l1': tensor([[ 0.1572,  0.0173, -0.1390],
         [-0.3501, -0.1036, -0.2324],
         [ 0.4665, -0.5620, -0.3683],
         [-0.4697, -0.1055,  0.4499],
         [ 0.5061, -0.5026, -0.0250]]),
 'l2': tensor([[ 0.2625,  0.1971,  0.3663, -0.0124, -0.3005],
         [ 0.2958, -0.1534,  0.2316, -0.1102,  0.1901],
         [ 0.4022,  0.4158, -0.4070, -0.2958, -0.1849]])}

In [277]:
optimizer = torch.optim.SGD([{"params": l2.parameters(), "lr": 10}, {"params": l1.parameters(), "lr": 10}])
optimizer

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 10
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0

Parameter Group 1
    dampening: 0
    differentiable: False
    foreach: None
    lr: 10
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [254]:
# перед backward pass попробуем проверить, есть ли какие-то градиенты 

In [278]:
l1.weight.grad

In [279]:
l2.weight.grad

In [ ]:
random_input -> l1 -> y -> l2 -> z

In [280]:
y = l1(random_input)
z = l2(y)

In [281]:
y

tensor([[ 0.5399, -0.0027,  1.6861, -1.6732,  0.9818],
        [ 0.2649,  0.2384,  1.3897, -0.7818,  0.8677],
        [ 0.1779, -0.0609, -0.0179, -0.6015, -0.1991]], grad_fn=<MmBackward0>)

In [282]:
z

tensor([[ 0.4846,  0.9216, -0.1568],
        [ 0.3746,  0.6147, -0.2891],
        [ 0.0954,  0.0862,  0.2683]], grad_fn=<MmBackward0>)

In [283]:
loss = loss_fn(z, random_output)

In [284]:
loss

tensor(0.6677, grad_fn=<MseLossBackward0>)

In [285]:
loss.backward()

In [286]:
l1.weight.grad

# как можно заметить, grad для слоя l1 расчитались т.к. requires_grad=True!

tensor([[ 2.6226e-01, -1.2790e-01, -4.4087e-01],
        [-2.4403e-04, -1.6964e-01, -7.9655e-02],
        [ 2.9802e-01, -2.3770e-02, -4.3012e-01],
        [-5.4779e-02,  4.4075e-02,  1.0432e-01],
        [-3.4147e-02,  1.6901e-01,  1.2350e-01]])

In [287]:
l2.weight.grad

In [288]:
optimizer.step()
optimizer.zero_grad()

In [266]:
# посмотрим на параметры слоёв, после backward step

In [289]:
params_after = {"l1": list(l1.parameters())[0].detach().clone(), "l2": list(l2.parameters())[0].detach().clone()}
params_after

{'l1': tensor([[-2.4654,  1.2964,  4.2697],
         [-0.3476,  1.5928,  0.5641],
         [-2.5137, -0.3243,  3.9329],
         [ 0.0781, -0.5463, -0.5933],
         [ 0.8475, -2.1927, -1.2600]]),
 'l2': tensor([[ 0.2625,  0.1971,  0.3663, -0.0124, -0.3005],
         [ 0.2958, -0.1534,  0.2316, -0.1102,  0.1901],
         [ 0.4022,  0.4158, -0.4070, -0.2958, -0.1849]])}

In [290]:
params_before

{'l1': tensor([[ 0.1572,  0.0173, -0.1390],
         [-0.3501, -0.1036, -0.2324],
         [ 0.4665, -0.5620, -0.3683],
         [-0.4697, -0.1055,  0.4499],
         [ 0.5061, -0.5026, -0.0250]]),
 'l2': tensor([[ 0.2625,  0.1971,  0.3663, -0.0124, -0.3005],
         [ 0.2958, -0.1534,  0.2316, -0.1102,  0.1901],
         [ 0.4022,  0.4158, -0.4070, -0.2958, -0.1849]])}

In [291]:
params_after["l1"] == params_before["l1"], params_after["l2"] == params_before["l2"]

(tensor([[False, False, False],
         [False, False, False],
         [False, False, False],
         [False, False, False],
         [False, False, False]]),
 tensor([[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]]))

In [ ]:
# Вывод: 
# Как можно заметить, градиенты для 2ого слоя не поступили, но при этом для 1ого слоя градиенты расчитаны

# Хоть 2ой слой и был подан в оптимизатор, обновления параметров не произошло т.к. слой был заморожен

### 5) 2 слоя, заморожён 1ый слой l1, подаём оба слоя оптимизаотру

In [292]:
# model
l1 = nn.Linear(3, 5, bias=False)
l2 = nn.Linear(5, 3, bias=False)

In [293]:
# замораживаем параметры 1ого слоя l1

for param in l1.parameters():
    param.requires_grad = False

In [294]:
params_before = {"l1": list(l1.parameters())[0].detach().clone(), "l2": list(l2.parameters())[0].detach().clone()}
params_before

{'l1': tensor([[-0.3682,  0.5540, -0.0510],
         [-0.3830, -0.2569, -0.3189],
         [ 0.2356, -0.5036, -0.5383],
         [-0.5691,  0.2486,  0.2855],
         [-0.3586,  0.1432,  0.4262]]),
 'l2': tensor([[ 0.0650, -0.0946,  0.0330, -0.0480,  0.2252],
         [-0.3037,  0.2268,  0.2521, -0.2710,  0.3652],
         [ 0.4392, -0.1763, -0.1253,  0.2229, -0.3504]])}

In [295]:
optimizer = torch.optim.SGD([{"params": l2.parameters(), "lr": 10}, {"params": l1.parameters(), "lr": 10}])
optimizer

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 10
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0

Parameter Group 1
    dampening: 0
    differentiable: False
    foreach: None
    lr: 10
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [254]:
# перед backward pass попробуем проверить, есть ли какие-то градиенты 

In [296]:
l1.weight.grad

In [297]:
l2.weight.grad

In [ ]:
random_input -> l1 -> y -> l2 -> z

In [298]:
y = l1(random_input)
z = l2(y)

In [299]:
y

tensor([[-0.6188,  0.1800,  1.6836, -1.5719, -1.5232],
        [-0.7388,  0.5070,  1.4330, -0.9923, -0.9552],
        [ 0.3524, -0.1404,  0.0602, -0.2266, -0.3312]])

In [300]:
z

tensor([[-0.2692,  0.5228, -0.3311],
        [-0.2161,  0.6206, -0.4799],
        [-0.0255, -0.1832,  0.2375]], grad_fn=<MmBackward0>)

In [301]:
loss = loss_fn(z, random_output)

In [302]:
loss

tensor(0.3013, grad_fn=<MseLossBackward0>)

In [303]:
loss.backward()

In [304]:
l1.weight.grad

# grad для слоя l1 не получен т.к. данный слой заморожён

In [307]:
l2.weight.grad

tensor([[-0.1352,  0.0822,  0.2522, -0.1806, -0.1705],
        [-0.0451, -0.0212,  0.2711, -0.3045, -0.3052],
        [-0.0371,  0.0503, -0.0055,  0.0557,  0.0582]])

In [308]:
optimizer.step()
optimizer.zero_grad()

In [266]:
# посмотрим на параметры слоёв, после backward step

In [309]:
params_after = {"l1": list(l1.parameters())[0].detach().clone(), "l2": list(l2.parameters())[0].detach().clone()}
params_after

{'l1': tensor([[-0.3682,  0.5540, -0.0510],
         [-0.3830, -0.2569, -0.3189],
         [ 0.2356, -0.5036, -0.5383],
         [-0.5691,  0.2486,  0.2855],
         [-0.3586,  0.1432,  0.4262]]),
 'l2': tensor([[ 1.4171, -0.9162, -2.4887,  1.7585,  1.9297],
         [ 0.1477,  0.4392, -2.4590,  2.7739,  3.4173],
         [ 0.8105, -0.6794, -0.0700, -0.3341, -0.9321]])}

In [310]:
params_before

{'l1': tensor([[-0.3682,  0.5540, -0.0510],
         [-0.3830, -0.2569, -0.3189],
         [ 0.2356, -0.5036, -0.5383],
         [-0.5691,  0.2486,  0.2855],
         [-0.3586,  0.1432,  0.4262]]),
 'l2': tensor([[ 0.0650, -0.0946,  0.0330, -0.0480,  0.2252],
         [-0.3037,  0.2268,  0.2521, -0.2710,  0.3652],
         [ 0.4392, -0.1763, -0.1253,  0.2229, -0.3504]])}

In [311]:
params_after["l1"] == params_before["l1"], params_after["l2"] == params_before["l2"]

(tensor([[True, True, True],
         [True, True, True],
         [True, True, True],
         [True, True, True],
         [True, True, True]]),
 tensor([[False, False, False, False, False],
         [False, False, False, False, False],
         [False, False, False, False, False]]))

In [ ]:
# Вывод: 
# Как можно заметить, градиенты для 1ого слоя не поступили, но при этом для 2ого слоя градиенты расчитаны

# Хоть 1ый слой и был подан в оптимизатор, обновления параметров не произошло т.к. слой был заморожен

### 6) 2 слоя, оба заморожёны - l1 и l2, подаём оба слоя оптимизатору

In [312]:
# model
l1 = nn.Linear(3, 5, bias=False)
l2 = nn.Linear(5, 3, bias=False)

In [313]:
# замораживаем параметры 1ого слоя l1

for param in l1.parameters():
    param.requires_grad = False
    
for param in l2.parameters():
    param.requires_grad = False

In [314]:
params_before = {"l1": list(l1.parameters())[0].detach().clone(), "l2": list(l2.parameters())[0].detach().clone()}
params_before

{'l1': tensor([[ 0.2283,  0.1347,  0.3639],
         [-0.5015, -0.4797,  0.2593],
         [ 0.2080, -0.5374, -0.3220],
         [-0.5283, -0.4136, -0.2605],
         [ 0.4778, -0.1149,  0.3316]]),
 'l2': tensor([[ 0.4247,  0.2038, -0.4004,  0.2250,  0.1706],
         [ 0.3466, -0.4411,  0.2750,  0.3661, -0.4161],
         [ 0.0468,  0.3818, -0.3253,  0.1467,  0.1976]])}

In [315]:
optimizer = torch.optim.SGD([{"params": l2.parameters(), "lr": 10}, {"params": l1.parameters(), "lr": 10}])
optimizer

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 10
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0

Parameter Group 1
    dampening: 0
    differentiable: False
    foreach: None
    lr: 10
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [254]:
# перед backward pass попробуем проверить, есть ли какие-то градиенты 

In [316]:
l1.weight.grad

In [317]:
l2.weight.grad

In [ ]:
random_input -> l1 -> y -> l2 -> z

In [318]:
y = l1(random_input)
z = l2(y)

In [319]:
y

tensor([[-0.4806, -1.1970,  1.1797, -0.1251,  0.0475],
        [-0.5253, -0.1275,  1.1513,  0.5126, -0.0676],
        [-0.0568, -0.7907, -0.1349, -0.3723, -0.1421]])

In [320]:
z

tensor([[-0.9404,  0.6204, -0.8721],
        [-0.6062,  0.4066, -0.3859],
        [-0.2393,  0.2149, -0.3434]])

In [321]:
loss = loss_fn(z, random_output)

In [322]:
loss

tensor(0.5722)

In [323]:
loss.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

# NOTE!

т.к. у нас нет обучаемых параметров, то мы просто не можем сделать backward pass т.к. нельзя вычислить производные

In [324]:
l1.weight.grad

# grad для слоя l1 не получен т.к. данный слой заморожён

In [307]:
l2.weight.grad

tensor([[-0.1352,  0.0822,  0.2522, -0.1806, -0.1705],
        [-0.0451, -0.0212,  0.2711, -0.3045, -0.3052],
        [-0.0371,  0.0503, -0.0055,  0.0557,  0.0582]])

In [308]:
optimizer.step()
optimizer.zero_grad()

In [266]:
# посмотрим на параметры слоёв, после backward step

In [309]:
params_after = {"l1": list(l1.parameters())[0].detach().clone(), "l2": list(l2.parameters())[0].detach().clone()}
params_after

{'l1': tensor([[-0.3682,  0.5540, -0.0510],
         [-0.3830, -0.2569, -0.3189],
         [ 0.2356, -0.5036, -0.5383],
         [-0.5691,  0.2486,  0.2855],
         [-0.3586,  0.1432,  0.4262]]),
 'l2': tensor([[ 1.4171, -0.9162, -2.4887,  1.7585,  1.9297],
         [ 0.1477,  0.4392, -2.4590,  2.7739,  3.4173],
         [ 0.8105, -0.6794, -0.0700, -0.3341, -0.9321]])}

In [310]:
params_before

{'l1': tensor([[-0.3682,  0.5540, -0.0510],
         [-0.3830, -0.2569, -0.3189],
         [ 0.2356, -0.5036, -0.5383],
         [-0.5691,  0.2486,  0.2855],
         [-0.3586,  0.1432,  0.4262]]),
 'l2': tensor([[ 0.0650, -0.0946,  0.0330, -0.0480,  0.2252],
         [-0.3037,  0.2268,  0.2521, -0.2710,  0.3652],
         [ 0.4392, -0.1763, -0.1253,  0.2229, -0.3504]])}

In [311]:
params_after["l1"] == params_before["l1"], params_after["l2"] == params_before["l2"]

(tensor([[True, True, True],
         [True, True, True],
         [True, True, True],
         [True, True, True],
         [True, True, True]]),
 tensor([[False, False, False, False, False],
         [False, False, False, False, False],
         [False, False, False, False, False]]))

### 7) 2 слоя, заморожён 2ой слой l2, подаём только слой l2 оптимизатору

In [358]:
# model
l1 = nn.Linear(3, 5, bias=False)
l2 = nn.Linear(5, 3, bias=False)

In [359]:

for param in l2.parameters():
    param.requires_grad = False

In [360]:
params_before = {"l1": list(l1.parameters())[0].detach().clone(), "l2": list(l2.parameters())[0].detach().clone()}
params_before

{'l1': tensor([[-0.0687, -0.2144, -0.1490],
         [-0.3149, -0.1710, -0.4956],
         [ 0.4321,  0.2358, -0.2912],
         [-0.1971,  0.2590, -0.2138],
         [ 0.1522,  0.1590, -0.3634]]),
 'l2': tensor([[ 0.1120,  0.0274,  0.3906, -0.1618, -0.1150],
         [ 0.3853, -0.3652,  0.0323,  0.4309, -0.2005],
         [ 0.4287,  0.3968,  0.4161, -0.0427,  0.4046]])}

In [361]:
optimizer = torch.optim.SGD([{"params": l2.parameters(), "lr": 10}])
optimizer

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 10
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [362]:
# перед backward pass попробуем проверить, есть ли какие-то градиенты 

In [363]:
l1.weight.grad

In [364]:
l2.weight.grad

In [ ]:
random_input -> l1 -> y -> l2 -> z

In [367]:
y = l1(random_input)
z = l2(y)

In [368]:
y

tensor([[ 0.2818,  0.6447,  1.2310,  0.0861,  0.9798],
        [ 0.4019,  0.6996,  0.3971, -0.0939,  0.4224],
        [-0.1002,  0.0862,  0.5813,  0.2914,  0.4560]], grad_fn=<MmBackward0>)

In [369]:
z

tensor([[ 0.4034, -0.2465,  1.2816],
        [ 0.1859, -0.2130,  0.7901],
        [ 0.1186, -0.0172,  0.4051]], grad_fn=<MmBackward0>)

In [370]:
loss = loss_fn(z, random_output)

In [371]:
loss

tensor(0.9513, grad_fn=<MseLossBackward0>)

In [372]:
loss.backward()

# NOTE!

backward pass удался т.к. есть обучаемые параметры в графе вычислений, хоть мы их и не подали в оптимизатор

In [376]:
l1.weight.grad

tensor([[ 0.3104, -0.1020, -0.5003],
        [ 0.2538, -0.3571, -0.5175],
        [ 0.4150, -0.3202, -0.7383],
        [-0.0908,  0.2419,  0.2282],
        [ 0.2011, -0.2525, -0.3997]])

In [377]:
l2.weight.grad

In [378]:
optimizer.step()
optimizer.zero_grad()

In [266]:
# посмотрим на параметры слоёв, после backward step

In [379]:
params_after = {"l1": list(l1.parameters())[0].detach().clone(), "l2": list(l2.parameters())[0].detach().clone()}
params_after

{'l1': tensor([[-0.0687, -0.2144, -0.1490],
         [-0.3149, -0.1710, -0.4956],
         [ 0.4321,  0.2358, -0.2912],
         [-0.1971,  0.2590, -0.2138],
         [ 0.1522,  0.1590, -0.3634]]),
 'l2': tensor([[ 0.1120,  0.0274,  0.3906, -0.1618, -0.1150],
         [ 0.3853, -0.3652,  0.0323,  0.4309, -0.2005],
         [ 0.4287,  0.3968,  0.4161, -0.0427,  0.4046]])}

In [380]:
params_before

{'l1': tensor([[-0.0687, -0.2144, -0.1490],
         [-0.3149, -0.1710, -0.4956],
         [ 0.4321,  0.2358, -0.2912],
         [-0.1971,  0.2590, -0.2138],
         [ 0.1522,  0.1590, -0.3634]]),
 'l2': tensor([[ 0.1120,  0.0274,  0.3906, -0.1618, -0.1150],
         [ 0.3853, -0.3652,  0.0323,  0.4309, -0.2005],
         [ 0.4287,  0.3968,  0.4161, -0.0427,  0.4046]])}

In [381]:
params_after["l1"] == params_before["l1"], params_after["l2"] == params_before["l2"]

(tensor([[True, True, True],
         [True, True, True],
         [True, True, True],
         [True, True, True],
         [True, True, True]]),
 tensor([[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]]))

In [ ]:
# l2 не обновился т.к. он заморожён и для него не были поданы градиенты
# l1 не обновился т.к. хоть и были расчитаны градиенты, он не был подан в оптимизатор для обновления своих весов

# Вывод: 
   
Если есть цепочка из слоёв или моделей в графе вычислений, а нужно, чтобы не все слои/модели обучались(не важно идут ли те или иные слои/модели раньше или позже по графу вычислений), то есть нескольно способов решения:
    
    1) Заморозка слоёв/моделей с помощью requires_grad = False и подача всех слоёв/моделей в оптимизатор - обновятлся лишь те параметры, где requires_grad = True
    
    2) Не делать заморозку слоёв/моделей, а подать в оптимизатор лишь те слои/модели, которые должны обучаться - градиенты будут считаться для всех слоёв/моделей, но обновиться смогут лишь те, которые были поданы в оптимизатор 